In [1]:
from ast import literal_eval
import re
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone


import numpy as np
import pandas as pd

random_seed = 42

In [2]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [3]:
test = pd.read_csv('test.csv',index_col = 0).drop(columns = 'Unnamed: 0.1')
train = pd.read_csv('train.csv',index_col = 0).drop(columns = 'Unnamed: 0.1')

In [4]:
train.head()

,ID,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,features
0,0,B000F83SZQ,"[0, 0]",5.0,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000,"[-1.2551e-01, 2.7282e-02, -3.9638e-02, -8.9909..."
1,1,B000F83SZQ,"[2, 2]",4.0,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400,"[ 3.6394e-01, -5.9243e-02, 1.0343e-01, -3.1970..."
2,2,B000F83SZQ,"[2, 2]",4.0,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600,"[ 2.8251e-01, -3.5420e-02, -4.2864e-01, 3.1264..."
3,3,B000F83SZQ,"[1, 1]",5.0,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000,"[-7.1293e-01, -2.7817e-01, 5.6167e-01, -6.9253..."
4,4,B000F83SZQ,"[0, 1]",4.0,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200,"[ 3.9603e-01, 5.1992e-01, -8.5361e-01, 7.7056e..."


In [57]:
test.head()

,ID,index,asin,helpful,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,features
0,0,491309,B00CB1OPI4,"[2, 2]",Juvenile. Loss of parents. The adults are alwa...,"02 17, 2014",A11TR2XF6H9XG0,Roger A Fischer,Kind of Harry Potter-ish,1392595200,"[-5.2195e-01, 5.7623e-02, 4.0095e-01, -3.8415e..."
1,1,491310,B00CB1OPI4,"[4, 4]",THE REMARKABLES is the first storyline in A.D....,"06 25, 2013",AW4NFCPN81CS3,Sandy S.,3.5 stars-super hero action,1372118400,"[-2.3566e-01, 4.9495e-02, 2.3776e-01, 3.8068e-..."
2,2,491311,B00CB1OPI4,"[1, 2]",This is the third book I have recently purchas...,"02 20, 2014",A3SSDN0LP1X6UA,texas rob,Unable to finish,1392854400,"[-1.2835e-01, 4.8343e-01, 2.8470e-02, 1.7314e-..."
3,3,491312,B00CB1XYV8,"[0, 0]","From the reviews I read here, I almost didn't ...","06 9, 2013",A3O2UZACDPCGZ5,Alan Arthur Katz,Loved It!,1370736000,"[ 1.0172e-01, 3.4241e-01, 6.5892e-01, 2.1418e-..."
4,4,491313,B00CB1XYV8,"[1, 1]",This is a difficult book to review for me. Tec...,"04 13, 2014",A3AP1WFXX6T2LJ,"Amazon Customer ""Deep Soul""",Interesting,1397347200,"[ 1.0386e+00, -7.1670e-02, 9.7345e-01, 5.1744e..."


In [58]:
train.shape, test.shape

((1000, 11), (1000, 11))

In [59]:
target_columns = list(set(train.columns) - set(test.columns))[0]
target_columns

'overall'

In [60]:
train['reviewerID'].nunique() , test['reviewerID'].nunique()

(827, 949)

In [61]:
reviewers = list(train['reviewerID'].unique())
reviewers += list(test['reviewerID'].unique())

len(set(reviewers))

1758

In [62]:
train = train.drop(columns = ['reviewerID', 'reviewerName'])
test = test.drop(columns = ['reviewerID', 'reviewerName'])

In [63]:
train['asin'].nunique(), test['asin'].nunique()

(93, 73)

In [64]:
items = list(train['asin'].unique())
items += list(test['asin'].unique())

len(set(items))

166

In [65]:
train = train.drop(columns = ['asin'])
test = test.drop(columns = ['asin'])

In [66]:
train = train.drop(columns = ['reviewTime'])
test = test.drop(columns = ['reviewTime'])

In [67]:
train['day'] = pd.to_datetime(train['unixReviewTime'], unit='s').dt.day
train['month'] = pd.to_datetime(train['unixReviewTime'], unit='s').dt.month
train['dayofweek'] = pd.to_datetime(train['unixReviewTime'], unit='s').dt.dayofweek

In [68]:
test['day'] = pd.to_datetime(test['unixReviewTime'], unit='s').dt.day
test['month'] = pd.to_datetime(test['unixReviewTime'], unit='s').dt.month
test['dayofweek'] = pd.to_datetime(test['unixReviewTime'], unit='s').dt.dayofweek

In [69]:
train = train.drop(columns = ['unixReviewTime'])
test = test.drop(columns = ['unixReviewTime'])

In [70]:
train['helpful'] = train['helpful'].apply(lambda row: literal_eval(row))
train['helpful'] = train['helpful'].apply(lambda row: row[0] - (row[1] - row[0]))

test['helpful'] = test['helpful'].apply(lambda row: literal_eval(row))
test['helpful'] = test['helpful'].apply(lambda row: row[0] - (row[1] - row[0]))

In [71]:
train

,ID,helpful,overall,reviewText,summary,features,day,month,dayofweek
0,0,0,5.0,I enjoy vintage books and movies so I enjoyed ...,Nice vintage story,"[-1.2551e-01, 2.7282e-02, -3.9638e-02, -8.9909...",5,5,0
1,1,2,4.0,This book is a reissue of an old one; the auth...,Different...,"[ 3.6394e-01, -5.9243e-02, 1.0343e-01, -3.1970...",6,1,0
2,2,2,4.0,This was a fairly interesting read. It had ol...,Oldie,"[ 2.8251e-01, -3.5420e-02, -4.2864e-01, 3.1264...",4,4,4
3,3,1,5.0,I'd never read any of the Amy Brewster mysteri...,I really liked it.,"[-7.1293e-01, -2.7817e-01, 5.6167e-01, -6.9253...",19,2,2
4,4,-1,4.0,"If you like period pieces - clothing, lingo, y...",Period Mystery,"[ 3.9603e-01, 5.1992e-01, -8.5361e-01, 7.7056e...",19,3,2
5,5,0,4.0,A beautiful in-depth character description mak...,Review,"[-9.6401e-02, 4.7949e-01, -3.5940e-02, -1.2871...",26,5,0
6,6,0,4.0,I enjoyed this one tho I'm not sure why it's c...,Nice old fashioned story,"[ 4.0585e-02, 2.6970e-01, -5.8642e-02, -1.6313...",10,6,1
7,7,1,4.0,Never heard of Amy Brewster. But I don't need ...,Enjoyable reading and reminding the old times,"[ 2.6343e-01, 3.5448e-01, -1.6254e-01, -6.6858...",22,3,5
8,8,0,5.0,Darth Maul working under cloak of darkness com...,Darth Maul,"[-3.8516e-01, 9.7285e-02, 5.3222e-01, -4.8072e...",11,10,4
9,9,0,4.0,This is a short story focused on Darth Maul's ...,"Not bad, not exceptional","[-2.9248e-01, -5.6352e-01, -2.4623e-01, 1.4141...",13,2,6


In [72]:
train['features'] = train['features'].apply(lambda row: np.array(literal_eval(row)))

test['features'] = test['features'].apply(lambda row: np.array(literal_eval(row)))


In [73]:
test.head()

,ID,index,helpful,reviewText,summary,features,day,month,dayofweek
0,0,491309,2,Juvenile. Loss of parents. The adults are alwa...,Kind of Harry Potter-ish,"[-0.52195, 0.057623, 0.40095, -0.38415, -0.036...",17,2,0
1,1,491310,4,THE REMARKABLES is the first storyline in A.D....,3.5 stars-super hero action,"[-0.23566, 0.049495, 0.23776, 0.38068, 0.16758...",25,6,1
2,2,491311,0,This is the third book I have recently purchas...,Unable to finish,"[-0.12835, 0.48343, 0.02847, 0.17314, -0.05816...",20,2,3
3,3,491312,0,"From the reviews I read here, I almost didn't ...",Loved It!,"[0.10172, 0.34241, 0.65892, 0.21418, 0.45376, ...",9,6,6
4,4,491313,1,This is a difficult book to review for me. Tec...,Interesting,"[1.0386, -0.07167, 0.97345, 0.51744, 0.0021222...",13,4,6


In [74]:
!pip install nltk

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stopwords_en = stopwords.words('english')

vectorizer =TfidfVectorizer(stop_words=[re.sub(r'^a-zA-Z_0-9\s', r'', sw) for sw in stopwords_en])
X_train = vectorizer.fit_transform(train['reviewText'].apply(lambda row : re.sub(r'a-zA-Z_0-9\s',r'', row)))
X_test = vectorizer.transform(test['reviewText'].apply(lambda row : re.sub(r'a-zA-Z_0-9\s',r'', row)))


In [76]:
y = train['overall']

In [77]:
from sklearn.naive_bayes import BernoulliNB , ComplementNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier,AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier

from sklearn.metrics import mean_absolute_error as mae

clf = RandomForestClassifier(n_estimators=10, max_depth = 100, random_state=34234)
clf2 = ExtraTreesClassifier(random_state=random_seed)
clf3 = AdaBoostClassifier(random_state=random_seed)

ber = BernoulliNB(alpha = 1)
com = ComplementNB(alpha= 0.1)
mn  = MultinomialNB(alpha = 0.01)
rc = RidgeClassifier(alpha = 0.1, tol = 1e-4, solver = 'auto', random_state=random_seed)



In [78]:
%%time
#RandomForest
clf.fit(X_train[:-200], y[:-200])

y_pred = clf.predict(X_train[-200:])
mae(y[-200:],y_pred)

Wall time: 234 ms


0.695

In [79]:
%%time
#Extra Trees
clf2.fit(X_train[:-200], y[:-200])

y_pred_2 = clf2.predict(X_train[-200:])
mae(y[-200:],y_pred_2)

Wall time: 3.98 s


0.695

In [80]:
%%time
#Ada Boost
clf3.fit(X_train[:-200], y[:-200])
y_pred_3 = clf3.predict(X_train[-200:])

mae(y[-200:],y_pred_3)

Wall time: 2.41 s


0.745

In [81]:
%%time
#Bernoulli Naive
ber.fit(X_train[:-200], y[:-200])

y_pred_ber = ber.predict(X_train[-200:])

mae(y[-200:],y_pred_ber)

Wall time: 36.8 ms


0.72

In [82]:
%%time
#Complement Naive
com.fit(X_train[:-200], y[:-200])

y_pred_com = com.predict(X_train[-200:])

mae(y[-200:],y_pred_com)

Wall time: 15.9 ms


0.95

In [83]:
%%time
#Multinominal Naive
mn.fit(X_train[:-200], y[:-200])

y_pred_mn = mn.predict(X_train[-200:])

mae(y[-200:],y_pred_mn)

Wall time: 25.8 ms


0.72

In [84]:
%%time
#Ridge Regression
rc.fit(X_train[:-200], y[:-200])

y_pred_rc = rc.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

Wall time: 316 ms


0.59

In [85]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [86]:
from xgboost import XGBClassifier ,XGBRFClassifier
from sklearn.ensemble import GradientBoostingClassifier
xgbc = XGBClassifier()
xgbc1 = XGBRFClassifier()
GBC = GradientBoostingClassifier()

In [87]:
xgbb = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


In [88]:
%%time
xgbb.fit(X_train[:-200], y[:-200])

y_pred_rc = xgbb.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

Wall time: 1min 11s


0.655

In [89]:
%%time
xgbc.fit(X_train[:-200], y[:-200])

y_pred_rc = xgbc.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

Wall time: 26 s


0.67

In [90]:
GBC.fit(X_train[:-200], y[:-200])

y_pred_rc = GBC.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

0.71

In [91]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [92]:
w1 = 0.25
w2 = 0.40
w3 = 0.25
w4 = 0.10


In [93]:
weight_avg = AverageWeight(mod = [xgbc,rc,xgbb,ker,ela,bay],weight=[w1,w2,w3,w4,w5,w6])

In [95]:
weight_avg.fit(X_train[:-200], y[:-200])
y_pred_rc = weight_avg.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

0.7122499999999999

In [96]:
y_pred_rc = xgbb.predict(X_train[-200:])

mae(y[-200:], y_pred_rc)

0.655

In [ ]:
submission1 = pd.DataFrame({
    'ID': test.ID,
    'overall': weight_avg.predict(X_test)
})

In [ ]:
submission1.to_csv('submission1.csv', index=None)